In [1]:
import pandas as pd
import numpy as np
from keras.layers import Activation, Dense, Dropout, SpatialDropout1D,Input,Masking,Bidirectional, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
from keras.models import Sequential, Model
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from random import seed
#seed(1)
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.metrics import roc_auc_score


#defining the early stopping criteria
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,restore_best_weights=True,patience=3)


Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
gs = pd.read_csv('data_8th_june.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15,16,17,25,32,33,35,38,39,41,42,43,44,45,46,47,48,53,55,57,68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
gs.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'uhid', 'modificationtime_x', 'starttime',
       'pulserate', 'ecg_resprate', 'spo2', 'heartrate', 'dischargestatus_x',
       'mean_bp', 'sys_bp', 'dia_bp', 'time_x', 'date', 'modificationtime_y',
       'start_time', 'creationtime_x', 'peep', 'pip', 'map', 'tidalvol',
       'minvol', 'ti', 'fio2', 'time_y', 'ttime', 'hour_series',
       'abd_difference', 'abd_difference_y', 'abdomen_girth_x',
       'abdomen_girth_y', 'actual_DOA', 'actual_DOB', 'centraltemp',
       'creationtime_y', 'currentdateheight', 'currentdateweight', 'day',
       'dischargeddate', 'dischargestatus_y', 'entrydate_x', 'entrydate_y',
       'entrydatetime', 'hour', 'medicationtype', 'medicinename',
       'medicineorderdate', 'modificationtime', 'new_ph', 'ph', 'rbs',
       'skintemp', 'startdate', 'stool_day_total', 'stool_passed', 'temp',
       'time', 'time_divide', 'total_intake', 'totalparenteralvolume',
       'tpn-tfl', 'typevalue_Antibiotics', 'typevalue_Inotro

In [4]:
cols_to_use = ['uhid','pulserate',  'ecg_resprate', 'spo2', 'heartrate','dia_bp','dischargestatus']

In [5]:
gd = gs[cols_to_use]

In [6]:
def range_finder(x):
    length = x
    fractional = (x/15.0) - math.floor(x/15.0)
    return int(round(fractional*15))

In [7]:
import math
final_df = pd.DataFrame(columns=gd.columns)
for i in gd.uhid.unique():
    x = gd[gd['uhid']==i]
    x = x[range_finder(len(x)):len(x)]
    
    final_df = final_df.append(x,ignore_index=True)
    

In [8]:
final_df.fillna(-999,inplace=True)

In [9]:
train = final_df[:515340]
test = final_df[515340:]

In [10]:
y_train = train['dischargestatus']
X_train = train.drop('dischargestatus',axis=1)
X_train = X_train.drop('uhid',axis=1)
#X_train = X_train.drop('visittime',axis=1)

y_test = test['dischargestatus']
X_test = test.drop('dischargestatus',axis=1)
X_test = X_test.drop('uhid',axis=1)
#X_test = X_test.drop('startdate',axis=1)

In [11]:
#converting the data into a numpy array
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

ytrain1 = []
for i in range(0,len(y_train),15):
    #print(i)
    y1 = y_train[i:i+15]
    ytrain1.append(y1[-1])
    
ytest1 = []
for i in range(0,len(y_test),15):
    #print(i)
    y1 = y_test[i:i+15]
    ytest1.append(y1[-1])
    
ytrain1 = np.array(ytrain1)
ytest1 = np.array(ytest1)

Xtrain = np.reshape(X_train, (-1, 15, X_train.shape[1]))
Xtest = np.reshape(X_test, (-1, 15, X_test.shape[1]))

In [12]:
#Building the LSTM model
X = Input(shape=(None, 5), name='X')
mX = Masking()(X)
lstm = Bidirectional(LSTM(units=512,activation='tanh',return_sequences=True,recurrent_dropout=0.5,dropout=0.3))
mX = lstm(mX)
L = LSTM(units=64,activation='tanh',return_sequences=False)(mX)
y = Dense(1, activation="sigmoid")(L)
outputs = [y]
inputs = [X]
model = Model(inputs,outputs)
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])


#fitting the model
model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
#predictions
y_pred = model.predict(Xtest)
#y_pred = y_pred.round()
y_test = np.array(ytest1)
y_pred = np.array(y_pred)
y_test = pd.DataFrame(y_test)
y_test = np.array(y_test)

def acc(x):
    if x>0.5:
        return 1
    else:
        return 0

y_model=[]
for i in y_pred:
    y_model.append(acc(i))
y_answer=[]
for j in y_test:
    y_answer.append(acc(j))

auc_roc = roc_auc_score(y_answer,y_pred)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 29202 samples, validate on 5154 samples
Epoch 1/38
29202/29202 [==============================] - 161s 6ms/step - loss: 0.1770 - acc: 0.9498 - val_loss: 0.3082 - val_acc: 0.8915
Epoch 2/38
29202/29202 [==============================] - 197s 7ms/step - loss: 0.1657 - acc: 0.9548 - val_loss: 0.3049 - val_acc: 0.8939
Epoch 3/38
29202/29202 [==============================] - 184s 6ms/step - loss: 0.1632 - acc: 0.9555 - val_loss: 0.3186 - val_acc: 0.8943
Epoch 4/38
29202/29202 [==============================] - 170s 6ms/step - loss: 0.1626 - acc: 0.9556 - val_loss: 0.3174 - val_acc: 0.8943
Epoch 5/38
29202/29202 [==============================] - 162s 6ms/step - loss: 0.1622 - acc: 0.9555 - val_loss: 0.3074 - val_acc: 0.8943
Restoring model weights from the end of the be

In [13]:
auc_roc

0.8738154272919683